<a href="https://colab.research.google.com/github/HaleBera/Container-Attack-Dataset/blob/main/VulfocusAutomate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some variables should be given as an input to the program:

1.   Sytem Path Info (String): '/home/hale/Desktop/TEST_ENVIRONMENT/Vulfocus/vulfocus_automater-master/vulfocusAutomate'
2.   File of Image List's Path Info (String): './readyToAttackImages.txt'
3.   File of Nuclei Attack List's Path Info (String): './readyToAttackNucleiTemplates.txt'
4.   File where Nuclei exports its results (String):'./nuclei_results'
     (We need a mini coding that will add the date-time info at the end of the results file name i.e. nuclei_results_10-04-2022_14:26)

 

Some coding segments are needed to be rewrited to be able to start multiple containers at the same time (Or if is not possible "at the same time", we may try to start and close one-by-one) 
*   ## START CONTAINER

      startedContainer = clinet.start_container(imageLine)
      
*   ## STOP AND DELETE CONTAINER
        #print(clinet.stop_container(imageLine))
        #print(clinet.delete_container(imageLine))



In [ ]:
import sys
sys.path.insert(0, '/home/hale/Desktop/TEST_ENVIRONMENT/Vulfocus/vulfocus_automater-master/vulfocusAutomate')

import os
VERBOSE = False
from src.vulfocus.vulfocusClient import VulfocusClient

import credentials
clinet = VulfocusClient(username=credentials.USERNAME,licence=credentials.LICENCE)
images = clinet.get_images()

# Input File Variables
imageFileToRead = open('./readyToAttackImages.txt', 'r')
nucleiTemplateFileToRead = open('./readyToAttackNucleiTemplates.txt', 'r')

if(VERBOSE):
    print(images)

# Main Loop
count = 0
while True:
    count += 1
 
    # Get next line from both files
    imageLine = imageFileToRead.readline()
    nucleiLine =  nucleiTemplateFileToRead.readline()
 
    # if line is empty
    # end of file is reached
    if not imageLine:
        break
    print("Starting image name: {} nuclei attack: {}".format(imageLine.strip(), nucleiLine.strip()))

    ## START CONTAINER
    startedContainer = clinet.start_container(imageLine)
    if(VERBOSE):
        print(startedContainer)
    startedHost = startedContainer.host
    stertedHostWithoutPort = startedHost.split(':')[0]
    startedPorts = startedContainer.port

    # we dont know which port is working for nuclei attack
    # why not try all :D
    for port in startedPorts.values():
        
        ## NUCLEI ATTACK STARTS HERE
        nucleiCommand = "sudo nuclei -u http://{} -t {} -debug -markdown-export ./nuclei_results".format(stertedHostWithoutPort +":"+ port, nucleiLine.strip())
        cwd = os.path.dirname(os.path.realpath(__file__))

        print("Switching to current working directory")
        commandResult1 = os.system("cd "+cwd)
        
        print("Running nuclei attack with port: {} :D Hope it will find something.".format(port))
        try:
            commandResult2 = os.system(nucleiCommand)
        except:
            print("Failed to execute nuclei command")
        ## NUCLEI ATTACK ENDS HERE

        ## STOP AND DELETE CONTAINER
        #print(clinet.stop_container(imageLine))
        #print(clinet.delete_container(imageLine))


